### install dependences

In [ ]:
pip install transformers

### tool: create image from a input mask

In [ ]:
from PIL import Image

def mask_image(mask, raw_image):
    weigth, heigth = raw_image.size
    new_image = Image.new('RGB', (weigth, heigth))
    for i in range (len(raw_image)):
        for j in range (len(raw_image[i])):
            if mask[i][j]:
                new_image[i][j] = raw_image[i][j]
            else:
                new_image[i][j] = 0
    return new_image

### Working with SAM

##### import dependences and model from hugginface

In [ ]:
import torch
from PIL import Image
import requests
from transformers import SamModel, SamProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"
model = SamModel.from_pretrained("facebook/sam-vit-huge").to(device)
processor = SamProcessor.from_pretrained("facebook/sam-vit-huge")

##### select and import image by process

In [ ]:
img_url = "https://huggingface.co/ybelkada/segment-anything/resolve/main/assets/car.png"

raw_image = Image.open(requests.get(img_url, stream=True).raw).convert("RGB")
input_points = [[[450, 600]]]  # 2D location of a window in the image

##### masks from image

In [ ]:
inputs = processor(raw_image, input_points=input_points, return_tensors="pt").to(device)
outputs = model(**inputs)

#Estas son mascaras booleanas True implica que el pixel en cuestion interesa, y False implica que no es de interes, es Fondo de esa capa
masks = processor.image_processor.post_process_masks(
    outputs.pred_masks.cpu(), inputs["original_sizes"].cpu(), inputs["reshaped_input_sizes"].cpu()
)

### Print images shortcuts. 
once we have a masks of the input image, for each mask we will print the image that is in this position

In [ ]:
import matplotlib.pyplot as plt

for i, mask in enumerate(masks):
    image_to_print = mask_image(mask, raw_image)
    image_to_print = image_to_print.squeeze().permute(1,2,0) 
    
    plt.figure(figsize=(10, 10))
    plt.imshow(image_to_print)
    plt.title(f"Segmentation Mask {i+1}")
    plt.show()